In [ ]:
import sklearn
import pandas as pd
import numpy as np
import nltk
import sklearn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import train_test_split
import re
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import pickle
from wordcloud import WordCloud

In [ ]:
full_reviewed_data_drop = pd.read_csv('../../input_data/full_datasets/fully_hand_labelled_texts.csv')

In [ ]:
indices = pd.read_csv('medrobertanl-haga_smoking_predictions.csv', delimiter=';')

In [ ]:
test_set = full_reviewed_data_drop.loc[indices['index']]

In [ ]:
train_set = full_reviewed_data_drop.loc[~full_reviewed_data_drop.index.isin(test_set.index)]

In [ ]:
train_set['Roken'].value_counts()

In [ ]:
test_set['Roken'].value_counts()

In [ ]:
train_set['Alcohol'].value_counts()

In [ ]:
test_set['Alcohol'].value_counts()

In [ ]:
train_set['Drugs'].value_counts()

In [ ]:
test_set['Drugs'].value_counts()

In [ ]:
test_set.text = test_set.text.str.replace("_x000D_\n", "\t")


In [ ]:
Corpus = test_set.copy()

In [ ]:
roken_corpus = Corpus[['text', 'Roken']]

In [ ]:
roken_corpus['text'] = roken_corpus.text.str.lower()

In [ ]:
roken_corpus['text'] = roken_corpus.text.str.replace("\t", "")

In [ ]:
def sm_previous_user_check(text):
    return "rookt niet meer" in text or "gestopt met roken" in text

def sm_current_user_check(text):
    if "rookt +" in text or "rookt+" in text:
        return True
    if "roker +" in text or "roker+" in text:
        return True
    if "roken +" in text or "roken+" in text:
        return True
    if "rookster +" in text or "rookster+" in text:
        return True
    if "rookt: ja" in text:
        return True
    if "roker: ja" in text or "rookster: ja" in text:
        return True
    if "rookt soms" in text or "rookt: soms" in text:
        return True
    if "roken ++" in text:
        return True
    return False

def sm_non_user_check(text):
    if "rookt -" in text or "rookt-" in text:
        return True
    if "roker -" in text or "roker-" in text:
        return True
    if "roken -" in text or "roken-" in text:
        return True
    if "rookster -" in text or "rookster-" in text:
        return True
    if "rookt: nee" in text:
        return True
    if "roker: nee" in text or "rookster: nee" in text:
        return True
    if "rookt nooit" in text or "rookt: nooit" in text:
        return True
    if "roken --" in text:
        return True
    return False
    
def apply_string_matching(row):
    text = row['text']
    if sm_previous_user_check(text):
        return "Voormalige gebruiker"
    if sm_current_user_check(text):
        return "Huidige gebruiker"
    if sm_non_user_check(text):
        return "Geen gebruiker"
    return "Niets gevonden"

roken_corpus['sm_label'] = roken_corpus.apply(lambda row: apply_string_matching(row), axis=1)

In [ ]:
roken_corpus

In [ ]:
print(np.mean(roken_corpus['sm_label'] == roken_corpus['Roken']))
cm = confusion_matrix(roken_corpus['Roken'], roken_corpus['sm_label'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

plt.show()
print(metrics.classification_report(roken_corpus['Roken'], roken_corpus['sm_label']))

In [ ]:
roken_corpus.to_csv('smoking_string_matching_predictions.csv')

In [ ]:
alcohol_corpus = Corpus[['text', 'Alcohol']]

In [ ]:
alcohol_corpus

In [ ]:
alcohol_corpus['text'] = alcohol_corpus.text.str.lower()

In [ ]:
alcohol_corpus['text'] = alcohol_corpus.text.str.replace("\t", "")

In [ ]:
alcohol_corpus['Alcohol'] = alcohol_corpus.Alcohol.str.replace("Voormalige gebruiker", "Geen gebruiker")

In [ ]:
def al_current_user_check(text):
    if "alcohol +" in text or "alcohol+" in text:
        return True
    if "alcohol: ja" in text:
        return True
    if "drinkt alcohol" in text:
        return True
    if "alcohol af en toe" in text:
        return True
    if "alcohol per week" in text or "alcohol week" in text:
        return True
    if "invloed van alcohol" in text:
        return True
    return False

def al_non_user_check(text):
    if "alcohol -" in text or "alcohol-" in text:
        return True
    if "alcohol: nee" in text:
        return True
    if "drinkt geen alcohol" in text:
        return True
    if "geen alcohol" in text:
        return True
    if "alcohol: geheel niet" in text:
        return True
    if "alcohol: geen" in text:
        return True
    return False
    
def apply_string_matching(row):
    text = row['text']
    if al_current_user_check(text):
        return "Huidige gebruiker"
    if al_non_user_check(text):
        return "Geen gebruiker"
    return "Niets gevonden"

alcohol_corpus['al_label'] = alcohol_corpus.apply(lambda row: apply_string_matching(row), axis=1)

In [ ]:
alcohol_corpus

In [ ]:
print(np.mean(alcohol_corpus['al_label'] == alcohol_corpus['Alcohol']))
cm = confusion_matrix(alcohol_corpus['Alcohol'], alcohol_corpus['al_label'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

plt.show()
print(metrics.classification_report(alcohol_corpus['Alcohol'], alcohol_corpus['al_label']))

In [ ]:
drugs_corpus = Corpus[['text', 'Drugs']]

In [ ]:
drugs_corpus['text'] = drugs_corpus.text.str.lower()

In [ ]:
drugs_corpus['text'] = drugs_corpus.text.str.replace("\t", "")

In [ ]:
drugs_corpus['Drugs'] = drugs_corpus.Drugs.str.replace("Voormalige gebruiker", "Geen gebruiker")

In [ ]:
def dr_current_user_check(text):
    if "drugs +" in text or "drugs+" in text:
        return True
    if "drugs: ja" in text:
        return True
    if "gebruikt drugs" in text:
        return True
    return False

def dr_non_user_check(text):
    if "drugs -" in text or "drugs-" in text:
        return True
    if "drugs: nee" in text:
        return True
    if "drugs niet" in text:
        return True
    if "geen drugs" in text:
        return True
    return False
    
def apply_string_matching(row):
    text = row['text']
    if dr_current_user_check(text):
        return "Huidige gebruiker"
    if dr_non_user_check(text):
        return "Geen gebruiker"
    return "Niets gevonden"

drugs_corpus['dr_label'] = drugs_corpus.apply(lambda row: apply_string_matching(row), axis=1)

In [ ]:
print(np.mean(drugs_corpus['dr_label'] == drugs_corpus['Drugs']))
cm = confusion_matrix(drugs_corpus['Drugs'], drugs_corpus['dr_label'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

plt.show()
print(metrics.classification_report(drugs_corpus['Drugs'], drugs_corpus['dr_label']))